In [1]:
# Import the required libraries.
# !pip install tensorflow
import os
import cv2

import numpy as np

import tensorflow as tf
import keras
import random
import matplotlib.pyplot as plt

# from moviepy.editor import *
%matplotlib inline
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings('ignore')

In [2]:
from keras.applications.vgg16 import VGG16

In [3]:
model= VGG16()

In [4]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [5]:
deepface_model = tf.keras.models.Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)


In [6]:
deepface_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [7]:
mod = tf.keras.models.load_model(r"C:\Users\Lenovo\Desktop\lbp_1\model1.h5")

In [8]:
mod.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 4096)              117479232 
                                                                 
 dense (Dense)               (None, 5)                 20485     
                                                                 
Total params: 117499717 (448.23 MB)
Trainable params: 117499717 (448.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
deepface_model = tf.keras.models.Model(inputs=mod.layers[0].input, outputs=mod.layers[-2].output)

In [10]:
deepface_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [11]:
def normalize(x):
    return x / np.sqrt(np.sum(np.multiply(x, x)))
def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [12]:
def process_img(image_path):
    test_image=tf.keras.preprocessing.image.load_img(image_path,target_size=(224,224))
    test_image=tf.keras.preprocessing.image.img_to_array(test_image)
    test_image=np.expand_dims(test_image,axis=0)
    return test_image

In [14]:
# Embedding_List = [] # list of embeding matrix after conversion from cropped faces to embedding matrix using deepface
# Name_List = []  # list of names corrospoing to cropped photos
# for folderPath in os.listdir('dataset'):
#     fpath=os.path.join('dataset',folderPath) #traversing through all folders in the dataset
#     for imgpath in os.listdir(fpath): #traversing through all images in the sub folders of dataset
#         imagepath=os.path.join(fpath,imgpath)
#         Embedding_List.append(deepface_model.predict(process_img(imagepath))[0])
#         Name_List.append(folderPath)

# #saving data
# # data = [Name_List, Embedding_List]

1/1 [==============================] - 0s 172ms/step


In [15]:
# np.save('embeddings1.npy', Embedding_List)

In [16]:
# np.save('name_list1.npy',Name_List)

In [13]:
name_list = np.load('name_list1.npy')

In [14]:
embedding_list = np.load('embeddings1.npy')

In [15]:
def predict(image):
    dist_list = [] #making distance list array
    R_embedding = deepface_model.predict(process_img(image))[0]
    for emb_db in embedding_list:
        euclidean_distance = findEuclideanDistance(normalize(R_embedding),normalize(emb_db))
        dist_list.append(euclidean_distance)
    min_dist = min(dist_list) #finding min distance
    print(min_dist)
    if min_dist<0.70: #condition for surity, this value is selected by us by oberving its value for differnet persons
        min_dist_idx = dist_list.index(min_dist)
        name = name_list[min_dist_idx]
        return name
    else:
        name  = 'unknown'
        return name

In [16]:
import cv2
from pathlib import Path

open_door = False
cascade_path = cv2.data.haarcascades + "haarcascade_frontalface_default.xml" # Path to the Haar Cascade classifier XML file

# Load the cascade
face_cascade = cv2.CascadeClassifier(cascade_path)

cam = cv2.VideoCapture(0)

while True:
    ret, frame = cam.read()

    if not ret:
        print("No image error...")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert image to grayscale

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)  # Draw rectangle around the face

        # Crop and save the face image
        crop_img = frame[y:y + h, x:x + w]
        Path('Test_face/').mkdir(parents=True, exist_ok=True)
        cv2.imwrite('Test_face/newface.jpg', crop_img)
        R_ImagePath = 'Test_face/newface.jpg'

        # Here you can add your prediction function
        predicted_name = predict(R_ImagePath)
        cv2.putText(frame, predicted_name, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2,
                    cv2.LINE_AA)  # Write name of the predicted person above the rectangle

    cv2.imshow('video', frame)

    # Keypress handling
    key = cv2.waitKey(1)

    if key == 32:
        # Space pressed
        print("Analysing Person")
        person = predicted_name

    elif key == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break

cam.release()
cv2.destroyAllWindows()


1/1 [==============================] - 1s 1s/step
0.048364945
1/1 [==============================] - 0s 140ms/step
0.07028128
1/1 [==============================] - 0s 121ms/step
0.04274363
1/1 [==============================] - 0s 127ms/step
0.08493031
1/1 [==============================] - 0s 134ms/step
0.08814401
1/1 [==============================] - 0s 113ms/step
0.07624124
1/1 [==============================] - 0s 133ms/step
0.04488761
1/1 [==============================] - 0s 112ms/step
0.038059447
1/1 [==============================] - 0s 126ms/step
0.037674837
1/1 [==============================] - 0s 115ms/step
0.03500562
1/1 [==============================] - 0s 121ms/step
0.038507506
1/1 [==============================] - 0s 114ms/step
0.056917906
1/1 [==============================] - 0s 118ms/step
0.051037032
1/1 [==============================] - 0s 127ms/step
0.056979414
1/1 [==============================] - 0s 130ms/step
0.06986295
1/1 [==============================]

1/1 [==============================] - 0s 129ms/step
0.19050556
1/1 [==============================] - 0s 145ms/step
0.026692778
1/1 [==============================] - 0s 151ms/step
0.034408823
1/1 [==============================] - 0s 157ms/step
0.0387665
1/1 [==============================] - 0s 141ms/step
0.031129465
1/1 [==============================] - 0s 138ms/step
0.03078209
1/1 [==============================] - 0s 159ms/step
0.02820381
1/1 [==============================] - 0s 135ms/step
0.031823296
1/1 [==============================] - 0s 138ms/step
0.0262285
1/1 [==============================] - 0s 153ms/step
0.03158915
1/1 [==============================] - 0s 147ms/step
0.03464244
1/1 [==============================] - 0s 137ms/step
0.028216068
1/1 [==============================] - 0s 138ms/step
0.027252285
1/1 [==============================] - 0s 141ms/step
0.027745225
1/1 [==============================] - 0s 156ms/step
0.025141248
1/1 [=============================

1/1 [==============================] - 0s 207ms/step
0.041759346
1/1 [==============================] - 0s 194ms/step
0.060936507
1/1 [==============================] - 0s 176ms/step
0.042193625
1/1 [==============================] - 0s 180ms/step
0.06315281
1/1 [==============================] - 0s 192ms/step
0.023114545
1/1 [==============================] - 0s 193ms/step
0.048013102
1/1 [==============================] - 0s 178ms/step
0.20353062
1/1 [==============================] - 0s 207ms/step
0.087911434
1/1 [==============================] - 0s 178ms/step
0.22978161
1/1 [==============================] - 0s 186ms/step
0.5142112
1/1 [==============================] - 0s 175ms/step
0.09106777
1/1 [==============================] - 0s 163ms/step
0.09262017
1/1 [==============================] - 0s 160ms/step
0.096267015
1/1 [==============================] - 0s 177ms/step
0.10678904
1/1 [==============================] - 0s 191ms/step
0.10415155
1/1 [=============================

1/1 [==============================] - 0s 160ms/step
0.08797296
1/1 [==============================] - 0s 154ms/step
0.09803973
1/1 [==============================] - 0s 145ms/step
0.21349846
1/1 [==============================] - 0s 155ms/step
0.09377424
1/1 [==============================] - 0s 159ms/step
0.08091149
1/1 [==============================] - 0s 146ms/step
0.076661125
1/1 [==============================] - 0s 152ms/step
0.10437083
1/1 [==============================] - 0s 145ms/step
0.116729185
1/1 [==============================] - 0s 155ms/step
0.077691
1/1 [==============================] - 0s 159ms/step
0.20956789
1/1 [==============================] - 0s 144ms/step
0.15372707
1/1 [==============================] - 0s 153ms/step
0.19203365
1/1 [==============================] - 0s 147ms/step
0.1016627
1/1 [==============================] - 0s 141ms/step
0.19587584
1/1 [==============================] - 0s 151ms/step
0.1980427
1/1 [==============================] - 0s 

1/1 [==============================] - 0s 148ms/step
0.25107193
1/1 [==============================] - 0s 134ms/step
0.20191412
1/1 [==============================] - 0s 147ms/step
0.22172382
1/1 [==============================] - 0s 136ms/step
0.2766633
1/1 [==============================] - 0s 133ms/step
0.28002802
1/1 [==============================] - 0s 148ms/step
0.29995087
1/1 [==============================] - 0s 137ms/step
0.2851716
1/1 [==============================] - 0s 137ms/step
0.2603891
1/1 [==============================] - 0s 137ms/step
0.30248454
1/1 [==============================] - 0s 130ms/step
0.30875453
1/1 [==============================] - 0s 149ms/step
0.30238107
1/1 [==============================] - 0s 143ms/step
0.29547417
1/1 [==============================] - 0s 144ms/step
0.27147606
1/1 [==============================] - 0s 152ms/step
0.350352
1/1 [==============================] - 0s 141ms/step
0.20577624
1/1 [==============================] - 0s 149

1/1 [==============================] - 0s 137ms/step
0.21978794
1/1 [==============================] - 0s 121ms/step
0.21561506
1/1 [==============================] - 0s 128ms/step
0.33785173
1/1 [==============================] - 0s 130ms/step
0.21519639
1/1 [==============================] - 0s 141ms/step
0.34194013
1/1 [==============================] - 0s 123ms/step
0.22598799
1/1 [==============================] - 0s 138ms/step
0.2696655
1/1 [==============================] - 0s 142ms/step
0.119423755
1/1 [==============================] - 0s 135ms/step
0.28922722
1/1 [==============================] - 0s 123ms/step
0.15223159
1/1 [==============================] - 0s 136ms/step
0.26633954
1/1 [==============================] - 0s 127ms/step
0.06882591
1/1 [==============================] - 0s 139ms/step
0.29579875
1/1 [==============================] - 0s 121ms/step
0.097895704
1/1 [==============================] - 0s 136ms/step
0.3581769
1/1 [==============================] - 0